In [29]:
import requests
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
cdc_raw = pd.read_csv("https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD")
can_raw = requests.get("https://api.covidactnow.org/v2/counties.json?apiKey=81d0e97ecec0406abf12c80d6cd8ec93")

C:\Users\seanmcclure\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
rows = []
for entry in can_raw.json():
    county = entry['actuals']
    county['county'] = entry['county']
    county['state'] = entry['state']
    county['fips'] = entry['fips']
    rows.append(county)
can = (
    pd.DataFrame.from_records(rows)[["vaccinationsInitiated", "vaccinationsCompleted", "state", "county", "fips"]]
    .rename(columns={"vaccinationsInitiated":"can_initiated", "vaccinationsCompleted":"can_completed"})
)

In [31]:
cdc = (
    cdc_raw.loc[:, ["Date", "FIPS", "Recip_County", "Recip_State", "Administered_Dose1_Recip", "Series_Complete_Yes"]]
    .rename(columns={"Recip_County":"county", "Administered_Dose1_Recip":"cdc_initiated","Recip_State":"state", "Series_Complete_Yes":"cdc_completed", "FIPS":"fips"})
    .loc[cdc_raw["Date"] == "07/07/2021"]
    .drop(columns={"Date"})
)


## CDC County Coverage

In [32]:
cdc_coverage = len(cdc.loc[
    (cdc["cdc_initiated"] != 0) & 
    (cdc["county"] != "Unknown County") & 
    (cdc["cdc_completed"] != 0)
])
print(f"CDC has full coverage for {cdc_coverage} counties")

CDC has full coverage for 2737 counties


## CDC missing counties

In [84]:
unknown = len(cdc.loc[cdc["county"] == "Unknown County"])
missing = len(cdc.loc[(cdc["cdc_initiated"] == 0)]) # add  & (cdc["cdc_completed"] == 0)] for neither init or complete 
print(f"CDC has no data for {missing} counties, and tracks {unknown} unknown counties")

CDC has no data for 491 counties, and tracks 58 unknown counties


In [36]:
def missing(var):
    missing = (
    cdc.loc[cdc[var] == 0]
    .loc[:, ["state"]]
    .groupby("state")
    .size()
    )
    total = cdc.groupby("state").size()
    compare = pd.concat([missing, total], axis=1).dropna()
    compare.columns = ["missing counties", "total counties"]
    compare["% missing"] = round(compare["missing counties"] / compare["total counties"],4)*100
    return compare.sort_values("% missing", ascending=False)

#### States with Counties missing 1+ dose data

In [48]:
missing_initiated = missing("cdc_initiated").reset_index()
missing_initiated

States with missing initiated values: 


,state,missing counties,total counties,% missing
0,HI,5.0,5,100.00
1,TX,254.0,254,100.00
2,NE,77.0,94,81.91
3,SD,41.0,67,61.19
4,NM,19.0,34,55.88
5,CO,33.0,65,50.77
6,DC,1.0,2,50.00
7,GA,27.0,160,16.88
8,OH,13.0,89,14.61
9,CA,8.0,59,13.56


#### States with Counties missing vaccine completed data

In [8]:
missing("cdc_completed")

States with missing completed values: 


,missing counties,total counties,% missing
state,,,
HI,5.0,5,100.00
TX,254.0,254,100.00
CA,8.0,59,13.56
VA,7.0,134,5.22


## Compare CDC and Dashboard Data

In [74]:
full = pd.merge(cdc, can, how="left", on=["fips", "state"]).query("fips != 'UNK'")

### Percent Differences
`percent_difference` is calculated as: \begin{equation*}
\Bigg(\frac{\mbox{cdc values} - \mbox{state values}}{\frac{\mbox{cdc values} + \mbox{state values}}{2}}\Bigg)*100
\end{equation*}

In [75]:
full = full.assign(
    initiated_difference=lambda x: x['cdc_initiated'] - x['can_initiated'],
    percent_init_difference=lambda x: x['initiated_difference'] / ((x['cdc_initiated'] + x['can_initiated'])/2) * 100
)

#### Summary of % difference

In [76]:
summary = (
    full.query("cdc_initiated != 0")
    .sort_values('percent_init_difference', ascending=True)["percent_init_difference"]
    .describe()
)
summary["median"] = full.query("cdc_initiated != 0")['percent_init_difference'].median()
pd.DataFrame(summary)

,percent_init_difference
count,2723.000000
mean,-9.187677
std,33.841904
min,-196.026640
25%,-6.709903
50%,-1.243863
75%,2.252984
max,182.182628
median,-1.243863


#### States with largest under-representation by CDC data
(states where state-dashboard values are larger than the cdc values by at least 5% difference)

In [87]:
d = (
    full.query("cdc_initiated != 0")
    .groupby("state")
    .mean()
    .sort_values('percent_init_difference', ascending=False)
    .reset_index()
    .drop(columns={"cdc_initiated", "cdc_completed", "can_initiated", "can_completed"})
)
high_discrepancies = d.loc[d["percent_init_difference"] < -5]
high_discrepancies

,state,initiated_difference,percent_init_difference
40,DC,-21566.000000,-5.025002
41,AR,-786.653333,-5.826224
42,IL,-3972.990196,-6.049368
43,KS,-209.409524,-6.581650
44,NM,-9061.785714,-17.030311
45,ND,-836.188679,-20.752600
46,VT,-6237.285714,-26.911735
47,WV,-5954.636364,-63.203486
48,GA,-12752.954545,-75.638116
49,VA,-14544.235772,-91.830364


In [88]:
f = full.query("cdc_initiated != 0")
f["percent_init_difference"] = f["percent_init_difference"].abs()
f = f.sort_values("percent_init_difference", ascending=False).head(250)
pd.DataFrame(f.groupby("state").size().sort_values(ascending=False))

<ipython-input-88-5bd7a7c2a0b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f["percent_init_difference"] = f["percent_init_difference"].abs()


,0
state,
GA,102
VA,84
WV,30
SD,10
VT,4
AZ,3
NC,3
ND,3
LA,2


## States to block:
#### States are detected if they are missing any counties, or have a mean % difference of < -5% 
(negative % diff indicates state dashboard values are larger than CDC values)

In [108]:
# states that we currently use CDC for and cannot block
# DC is not missing a county, so do not block
states_to_not_block = ["CO", "FL", "DC", "MA", "AR"]

block = (
    pd.concat(
        [
            pd.DataFrame(missing_initiated["state"]).assign(explanation="missing counties"),
            pd.DataFrame(high_discrepancies["state"]).assign(explanation="% difference of < -5% (CDC under-counting)")
        ]
    )
    .groupby("state")['explanation']
    .apply(lambda x: ' and '.join(x))
    .reset_index()
    .query("state not in @states_to_not_block")
)
block

,state,explanation
1,CA,missing counties
5,GA,missing counties and % difference of < -5% (CD...
6,HI,missing counties
7,IL,% difference of < -5% (CDC under-counting)
8,KS,% difference of < -5% (CDC under-counting)
9,MI,missing counties
10,MT,missing counties
11,ND,% difference of < -5% (CDC under-counting)
12,NE,missing counties
13,NJ,missing counties
